In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag, map_tag
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
pf_df = pd.read_csv('kaggle_poem_dataset.csv')
pf_df['Author'] = pf_df.Author.astype('category')
pf_df.columns = [name.lower() for name in pf_df.columns]
pf_df.info()
pf_df = pf_df.drop(['unnamed: 0', 'poetry foundation id'], axis=1)
pf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15652 entries, 0 to 15651
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   unnamed: 0            15652 non-null  int64   
 1   author                15652 non-null  category
 2   title                 15652 non-null  object  
 3   poetry foundation id  15652 non-null  int64   
 4   content               15652 non-null  object  
dtypes: category(1), int64(2), object(2)
memory usage: 674.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15652 entries, 0 to 15651
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   author   15652 non-null  category
 1   title    15652 non-null  object  
 2   content  15652 non-null  object  
dtypes: category(1), object(2)
memory usage: 430.2+ KB


In [3]:
# create function to clean lines
import string, re
from nltk.corpus import words
import multiprocessing as mp
def clean_text(text):
    cont_dict = { 
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
        }

#     stopwords_list = stopwords.words("english")
    try:
        text = text.lower()
        for word in cont_dict.keys():
            pattern = re.compile(f'({str(word)})',re.I)
            text = pattern.sub(cont_dict[word],text)
        tokens = text.split()
#         text = ' '.join([word for word in tokens if word not in stopwords_list])
        text = ' '.join([''.join([c if c.isalnum() else ' ' for c in word]) for word in tokens])
        text = re.sub(' +', ' ', text)
        text = text.replace('\%','')
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('https?://\S+|www\.\S+', '', text)
        text = re.sub('<.*?>+', '', text)
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub('\n', ' ', text)
        text = re.sub('\w*\d\w*', '', text)
        text = " ".join(filter(lambda x:x[0]!="@", tokens))
        return text.strip()
    
    except AttributeError:
        text = str(text)
        return clean_text(text)
    
    except TypeError:
        text = text.decode()[1:]
        return clean_list(text.strip())
    return text

def clean_list(list_of_strings):
    return [clean_text(string) for string in list_of_strings]

In [ ]:
# create column to store document as list of lines
import multiprocessing as mp
pf_df['line'] = pf_df.content.str.split('\n')
with mp.Pool(processes=mp.cpu_count()-1) as pool:
    clean_lines = pool.map(clean_list, pf_df.line)
# pf_df['line'] = pf_df.line.map(clean_list)
pf_df['line'] = clean_lines
print(pf_df.iloc[0])

In [ ]:
# construct function to remove empty lines for accurate line count
def remove_empty_lines(list_of_lines):
    try:
        return [i for i in list_of_lines if i]
    except TypeError:
        return pd.NA
    
pf_df['line'] = pf_df.line.map(remove_empty_lines)
print(pf_df.iloc[0])

In [ ]:
# create column for length of poem in lines
pf_df['length_in_lines'] = pf_df.line.map(len)
pf_df.info()

In [ ]:
sns.displot(pf_df.length_in_lines)

In [ ]:
# filtering outliers with zscore
from scipy import stats
clean_df = pf_df[(np.abs(stats.zscore(pf_df.length_in_lines)) < 2.5)]

In [ ]:
# filter sub-haiku length works
clean_df = clean_df[clean_df.length_in_lines > 2]

In [ ]:
sns.displot(clean_df.length_in_lines, kde=True)

In [ ]:
def lexical_diversity(line_list):
    doc_string = ' '.join(line_list)
    words = word_tokenize(doc_string)
    return (len(set(words))/len(words))
        
clean_df['lexical_diversity'] = clean_df.line.map(lexical_diversity)

In [ ]:
clean_df.head()
with open('clean_df.pkl', 'wb') as f:
    pickle.dump(clean_df, f)

In [ ]:
sns.displot(clean_df.lexical_diversity, kde=True)

In [ ]:
# sample 3 lines from each poem
# import random
# clean_df.line.describe()
# docs_in_lines = clean_df.line.to_list()
# line_samples = [random.sample(lines,3) for lines in docs_in_lines]

In [ ]:
line_df = clean_df.explode('line').drop(['length_in_lines', 'content','lexical_diversity'],axis=1)

In [ ]:
line_df.info()

In [ ]:
line_df['words'] = line_df.line.map(word_tokenize)
line_df.head()

In [ ]:
line_df['length_in_words'] = line_df.words.map(len)

In [ ]:
line_df.describe()

In [ ]:
x = line_df.length_in_words
sns.displot(x, kde=True)

In [ ]:
# filtering outliers with zscore
from scipy import stats
clean_line_df = line_df[(np.abs(stats.zscore(line_df.length_in_words)) < 2.5)]

In [ ]:
# fig, ax = plt.subplots()
x = clean_line_df.length_in_words
sns.displot(x, kde=True)
# plt.tight_layout()

In [ ]:
line_df.info()

In [ ]:
clean_line_df.info()

In [ ]:
clean_line_df['lexical_diversity'] = clean_line_df.words.map(lexical_diversity)

In [ ]:
clean_line_df.head()
with open('clean_line_df.pkl', 'wb') as f:
    pickle.dump(clean_line_df, f)

In [ ]:
sns.displot(clean_line_df.lexical_diversity, kde=True)

In [ ]:
author_line_lexical = clean_line_df.drop(['title','line','words'], axis=1).groupby('author').mean()

In [ ]:
author_line_lexical.reset_index(inplace=True)
sns.displot(data=author_line_lexical, x='length_in_words',y='lexical_diversity')

In [ ]:
author_line_lexical.describe()

In [ ]:
# author_line_lexical_mode = clean_line_df.drop(['title','line','words'], axis=1).groupby('author').mode()
# author_line_lexical_mode.reset_index(inplace=True)
# sns.displot(data=author_line_lexical_avg, x='length_in_words',y='lexical_diversity')